In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install Augmentor


In [4]:
import Augmentor

In [24]:
p = Augmentor.Pipeline("/content/drive/MyDrive/Notebook/train/gum")

Initialised with 2 image(s) found.
Output directory set to /content/drive/MyDrive/Notebook/train/gum/output.

In [25]:
p.flip_left_right(0.5)
p.black_and_white(0.1)
p.rotate(0.3, 10, 10)
p.skew(0.4, 0.5)
p.zoom(probability = 0.2, min_factor = 1.1, max_factor = 1.5)
p.sample(5)

Processing <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=300x168 at 0x7FF9DAC51750>: 100%|██████████| 5/5 [00:00<00:00, 57.93 Samples/s]


In [26]:
p.flip_left_right(0.5)
p.black_and_white(0.1)
p.rotate(0.3, 10, 10)
p.skew(0.4, 0.5)
p.zoom(probability = 0.2, min_factor = 1.1, max_factor = 1.5)
p.sample(5)

Processing <PIL.Image.Image image mode=RGB size=300x168 at 0x7FF9DAE803D0>: 100%|██████████| 5/5 [00:00<00:00, 56.93 Samples/s]


In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [44]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Notebook/train'
valid_path = '/content/drive/MyDrive/Notebook/test'

In [45]:
inception=InceptionV3(include_top=False, weights='imagenet',input_shape=IMAGE_SIZE+[3])

In [46]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [47]:
folders = glob('/content/drive/MyDrive/Notebook/train')

In [48]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

In [49]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [50]:
# view the structure of the model
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 111, 111, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 111, 111, 32) 0           batch_normalization_188[0][0]    
____________________________________________________________________________________________

In [56]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)

In [57]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [58]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Notebook/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 30 images belonging to 3 classes.


In [59]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Notebook/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 30 images belonging to 3 classes.


In [60]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set))


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 1.0986 - accuracy: 0.3333 - val_loss: 16.3715 - val_accuracy: 0.5111
Epoch 2/10
1/1 [==============================] - 1s 565ms/step - loss: 0.4066 - accuracy: 0.9111 - val_loss: 12.6742 - val_accuracy: 0.5778
Epoch 3/10
1/1 [==============================] - 1s 565ms/step - loss: 7.1915e-05 - accuracy: 1.0000 - val_loss: 12.6531 - val_accuracy: 0.5778
Epoch 4/10
1/1 [==============================] - 1s 570ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 13.8671 - val_accuracy: 0.5333
Epoch 5/10
1/1 [==============================] - 1s 582ms/step - loss: 3.2384e-06 - accuracy: 1.0000 - val_loss: 13.8670 - val_accuracy: 0.5333
Epoch 6/10
1/1 [==============================] - 1s 582ms/step - loss: 8.2199e-04 - accuracy: 1.0000 - val_loss: 13.9229 - val_accuracy: 0.5333
Epoch 7/10
1/1 [==============================] - 1s 563ms/step - loss: 2.1298e-06 - accuracy: 1.0000 - val_loss: 13.9222 - val_accuracy: 0.5333


In [61]:
from tensorflow.keras.models import load_model

model.save('model_inception.h5')

In [63]:
y_pred = model.predict(test_set)

In [64]:
y_pred

array([[1.00000000e+00, 1.64332967e-20, 1.54132722e-15],
       [6.12489792e-09, 2.15349006e-04, 9.99784648e-01],
       [1.00000000e+00, 3.70750285e-14, 2.46682952e-10],
       [1.00000000e+00, 3.99875276e-15, 1.26917279e-11],
       [1.00000000e+00, 1.04377733e-16, 5.24756101e-17],
       [8.54719183e-05, 3.22664157e-03, 9.96687949e-01],
       [4.83279427e-06, 2.33158275e-01, 7.66836882e-01],
       [4.10327176e-03, 2.89690458e-08, 9.95896697e-01],
       [1.00000000e+00, 1.38587695e-13, 3.41093570e-10],
       [1.00000000e+00, 5.95107459e-14, 4.28225706e-13],
       [6.95911012e-07, 1.44947539e-04, 9.99854326e-01],
       [7.09643841e-01, 2.72890002e-01, 1.74661111e-02],
       [1.00000000e+00, 1.52463708e-12, 5.32681499e-09],
       [1.00000000e+00, 2.14969265e-20, 3.49563298e-15],
       [9.90802109e-01, 7.29908294e-04, 8.46796762e-03],
       [1.00000000e+00, 5.76698238e-20, 8.85080312e-14],
       [6.73576128e-09, 2.69817479e-04, 9.99730170e-01],
       [1.66163375e-07, 2.22103

In [65]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [66]:
y_pred

array([0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2,
       0, 2, 2, 0, 2, 0, 0, 2])